In [1]:
!pip install determined

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.7/563.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
!git clone https://github.com/determined-ai/determined

Cloning into 'determined'...
remote: Enumerating objects: 126308, done.
remote: Counting objects: 100% (1461/1461), done.
remote: Compressing objects: 100% (762/762), done.
remote: Total 126308 (delta 873), reused 1140 (delta 669), pack-reused 124847
Receiving objects: 100% (126308/126308), 121.63 MiB | 25.53 MiB/s, done.
Resolving deltas: 100% (97195/97195), done.


In [3]:
!python -m pip install --user virtualenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 40.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
!pip install tensorflow
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


 Download pre trained Model and dataset
---


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from PIL import Image
import requests
from io import BytesIO





# Visualize the dataset
---

In [7]:
# Load the CSV data
df = pd.read_csv('Cars damage1.csv')
df.head()

,url,Type_of_damage
0,bumper dent.png,bumper_dent
1,bumper scratch.png,bumper_scratch
2,door dent.png,door_dent
3,glass shatter.png,glass_shatter
4,broken headlamp.png,broken_headlamp


# load the pre-trained VGG16 model

In [9]:
# Encode the string labels as integers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Type_of_damage'] = le.fit_transform(df['Type_of_damage'])

model = VGG16(weights='imagenet', include_top=False)

# Extract features from each image and store them in a list
features = []
for url in df['url']:
    img = image.load_img(url, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(model.predict(x))

# Convert the list of features to a numpy array
X = np.concatenate(features)

# Convert the labels to a numpy array
#y = df['Type_of_damage'].values
# Convert the labels to a numpy array
y = df['Type_of_damage'].values.astype('float32')



1/1 [==============================] - 0s 491ms/step


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)



# Define and train a classifier on the extracted features
clf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(7, 7, 512)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
clf.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
1/1 [==============================] - 1s 980ms/step - loss: -14.9827 - accuracy: 0.0000e+00 - val_loss: -24.4553 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 54ms/step - loss: -323.7603 - accuracy: 0.0000e+00 - val_loss: -43.2123 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 53ms/step - loss: -514.9020 - accuracy: 0.0000e+00 - val_loss: -59.8610 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 48ms/step - loss: -777.2222 - accuracy: 0.0000e+00 - val_loss: -77.1807 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 49ms/step - loss: -958.8430 - accuracy: 0.0000e+00 - val_loss: -94.8861 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 57ms/step - loss: -1262.1869 - accuracy: 0.0000e+00 - val_loss: -112.6408 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 52ms/step - loss: -1344.2548 - ac

In [31]:
# Evaluate the classifier on the testing set
loss, accuracy = clf.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

1/1 [==============================] - 0s 28ms/step - loss: -801.8197 - accuracy: 0.0000e+00
Test accuracy: 0.0


In [39]:
# Load the CSV data
df_test = pd.read_csv('Car damage_test.csv')

# Encode the string labels as integers
df_test['Type of damage'] = le.fit_transform(df_test['Type of damage'])


# Extract features from each image and store them in a list
features_test = []
for url in df_test['url']:
    img = image.load_img(url, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features_test.append(model.predict(x))

# Convert the list of features to a numpy array
X_test_new = np.concatenate(features_test)

# Convert the labels to a numpy array
y_test_new = df_test['Type of damage'].values.astype('float32')

# Use the trained classifier to predict the class labels for the new images
y_pred = clf.predict(X_test_new)

# Convert the predicted probabilities to class labels
y_pred_class = (y_pred > 0.5).astype('float32')

# Evaluate the accuracy of the classifier on the new images
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_new, y_pred_class)
print('Test accuracy:', accuracy)


1/1 [==============================] - 0s 60ms/step
Test accuracy: 0.3333333333333333
